In [2]:
import os
from datetime import datetime
from dateutil.parser import parse
import time

import numpy as np
import pandas as pd
from scipy import interpolate
from scipy.optimize import curve_fit
import scipy.stats as scs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint
from tqdm import tqdm

import seaborn as sns
sns.set_theme(style="whitegrid")

from src.nft_analytics import NFTAnalytics
from src.ethereum_api import EthereumAPI

# Great Apes

In [ ]:
gaa = NFTAnalytics("0xA0F38233688bB578c0a88102A95b846c18bc0bA7")
DATA_FOLDER, RESULT_FOLDER = gaa.make_directories("GreatApeSociety")
TIMESTAMP = datetime.now().strftime("%Y-%m-%d_%H-%M")

In [ ]:
asset_data = gaa.load_json(filename=os.path.join(DATA_FOLDER, "data_great_apes.json"))

In [ ]:
asset_data = gaa.fetch_data(max_offset=10000)
gaa.save_json(asset_data, filename=os.path.join(DATA_FOLDER, "data_great_apes.json"))

In [ ]:
df = gaa.get_nft_holdings(asset_data, "GreatApes", eth_balances=False)

In [ ]:
df = df[df["GreatApes"] >= 5].sort_values("GreatApes", ascending=False)

In [ ]:
df.to_excel(os.path.join(RESULT_FOLDER, TIMESTAMP + "ape_holders_addresses_over_and_equal_10_holdings.xlsx"), index=False)

# Great Ape Bananas

In [3]:
gab = NFTAnalytics("0x4238e50bD593119Bbd0bAeD05f9851Eb84A0b0B7")
DATA_FOLDER, RESULT_FOLDER = gab.make_directories("GreatApeBananas")
TIMESTAMP = datetime.now().strftime("%Y-%m-%d_%H-%M")

2021-10-12 18:38:37,792 - src.ethereum_api - INFO - Connected to Infura endpoint.


In [ ]:
asset_data = gab.fetch_data(max_offset=5129)
gab.save_json(asset_data, filename=os.path.join(DATA_FOLDER, "data_great_ape_bananas.json"))

In [4]:
asset_data = gab.load_json(filename=os.path.join(DATA_FOLDER, "data_great_ape_bananas.json"))

In [22]:
diamond_addresses = []
standard_addresses = []

for asset in asset_data:
    for traits in asset["traits"]:
        if traits["trait_type"] == "Weapon":
            if "diamond" in traits["value"].lower():
                diamond_addresses.append(asset["owner"]["address"])
            else:
                standard_addresses.append(asset["owner"]["address"])

In [23]:
diamond_nfts_held = {}
standard_nfts_held = {}

for addresses in diamond_addresses:
    diamond_nfts_held[addresses] = 0

for addresses in diamond_addresses:
    diamond_nfts_held[addresses] += 1
    
for addresses in standard_addresses:
    standard_nfts_held[addresses] = 0

for addresses in standard_addresses:
    standard_nfts_held[addresses] += 1

In [16]:
df = pd.DataFrame(columns=["Address", "GreatApeBananasDiamond"])
for idx, (address, num_nfts) in enumerate(diamond_nfts_held.items()):
    df.loc[idx] = [address, num_nfts]

In [18]:
df = df.sort_values("GreatApeBananasDiamond", ascending=False)

In [19]:
df

,Address,GreatApeBananasDiamond
14,0xb104371d5a2680fb0d47ea9a3aa2348392454186,19
16,0xe2e8fee9ac2af4352a0e424650b77da25c36bf80,18
287,0xe26b5628da876589fe4127e03ea72a21b6d3f26f,7
125,0x713be5935b1f7cce777801c7432d46dcd7bde63e,7
69,0xc916ad19b24fc5830ced35e335f4475b148f1181,7
...,...,...
154,0xe30a9d6c47600c1e5ba522d2358ec7107f3335b9,1
155,0xa65c496f0bd5e9eaabbe7a6fe39a6119f064695c,1
156,0xdaf1a3bfab649bf402bdf7727a29166614199eba,1
157,0x5f4f382592f444203599422b47c4ff56417a9638,1


In [24]:
df = pd.DataFrame(columns=["Address", "GreatApeBananasStandard"])
for idx, (address, num_nfts) in enumerate(standard_nfts_held.items()):
    df.loc[idx] = [address, num_nfts]

In [26]:
df[df["GreatApeBananasStandard"] > 20]

,Address,GreatApeBananasStandard
1,0x4060d22091a82c7b8ff13c7baad7873cb2127807,21
14,0xab95a0fb53df696a492a93f30fa126bde2493043,22
15,0x0d85a368ace9d01bed5c229d2d55b807edda507f,28
17,0x8734b70b65511c6bfd0a36d4c44c595087577b2f,32
31,0x2b81eaac4936c5413bbd8536d6998e73e631bf53,25
32,0x2da4f2be3527cff7c5e23a4f495057c3addb1d04,35
38,0xd995104e7a5f992ffe0ea6d51d7d7c78f6b56512,40
39,0xb104371d5a2680fb0d47ea9a3aa2348392454186,29
41,0xc916ad19b24fc5830ced35e335f4475b148f1181,43
54,0x9395312a2e127e1a8c5b1da38f0af66350a16eb5,21
